### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from pathlib import Path

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
sys.path.append('/home/k64835/Master-Thesis-SITS')

scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [6]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.pretrained_temporal_feature_extraction import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from evaluation_scripts.patch_evaluation_helper import get_clustering_accuracy
from model_scripts.clustering import *
from model_scripts.subpatch_extraction import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.evaluation_helper import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## Pre-trained Model 1: Resnet 3D trained on Video Data (Temporal)

### Dataset prep: RGB

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)

Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

In [33]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, images_visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, images_visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

Create Patch-level Data Loaders

In [34]:
dataloader_train = create_data_loader(patch_tensor_train, field_numbers_train, batch_size=config.ae_batch_size, shuffle=True)
dataloader_eval = create_data_loader(patch_tensor_eval, field_numbers_eval, batch_size=config.ae_batch_size, shuffle=False)

### Extract Features

In [35]:
device = 'cuda'
resnet3D_extractor = ResNet3DFeatureExtractor()

tf_features_train, train_coord_dl = extract_features(resnet3D_extractor, dataloader_train, device)
tf_features_eval, eval_coord_dl = extract_features(resnet3D_extractor, dataloader_eval, device)

print("ResNet3D Extracted Features size:", tf_features_train.shape[1])  

ResNet3D Extracted Features size: 512


### K-means

In [36]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=11)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [37]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [38]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 1
Accuracy: 42.62
Precision: 50.0
Recall: 62.86
F1-score: 55.7
F2-score: 59.78


Save Model

In [39]:
# with open(config.resnet3D_path, 'wb') as file:
#     pickle.dump(kmeans, file)

## Pre-trained Model 2: Resnet trained on Sentinel-2 (Spectral)

### Dataset prep: B10 
Use patch level images

In [ ]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'b10')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

Get Last Timestep images since this model requires non-temporal data

In [ ]:
train_last_timestep_patches = get_last_timestep_patches(patch_tensor_train)
test_last_timestep_patches = get_last_timestep_patches(patch_tensor_eval)

test_last_timestep_patches.shape

torch.Size([48, 10, 64, 64])

Create Patch-level Data Loaders

In [ ]:
dataloader_train = create_data_loader(train_last_timestep_patches, field_numbers_train, batch_size=config.batch_size, shuffle=True)
dataloader_eval = create_data_loader(test_last_timestep_patches, field_numbers_eval, batch_size=config.batch_size, shuffle=False)

In [ ]:
for ip,_ in dataloader_eval:
    print(ip.shape)
    break

torch.Size([48, 10, 64, 64])


### Extract Features

In [ ]:
device = 'cuda'
time_steps = 7
in_channels = 10
sentinel2_resnet_extractor = SpectralSentinel2FeatureExtractor(in_channels)

tf_features_train, train_coord_dl = extract_features(sentinel2_resnet_extractor, dataloader_train, device)
tf_features_eval, eval_coord_dl = extract_features(sentinel2_resnet_extractor, dataloader_eval, device)

print("Resnet 18 (trained on Sentinel-2 images) Extracted Features Shape:", tf_features_train.shape[1])  

Resnet 18 (trained on Sentinel-2 images) Extracted Features Shape: 512


### K-means

In [ ]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=13)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [ ]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [ ]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 55.74
Precision: 60.53
Recall: 65.71
F1-score: 63.01
F2-score: 64.61


Save Model

In [ ]:
# with open(config.resent_sentinel_path, 'wb') as file:
#     pickle.dump(kmeans, file)

## Pre-trained Model 3: ViT pre-trained on ImageNet

### Dataset prep: RGB Images

Use patch-level images

In [45]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'rgb')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'rgb')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

In [46]:
resized_tensor_train = resize_images_transfer_learning(patch_tensor_train, (224, 224))
resized_tensor_eval = resize_images_transfer_learning(patch_tensor_eval, (224, 224))
resized_tensor_train.shape, resized_tensor_eval.shape

(torch.Size([2425, 7, 3, 224, 224]), torch.Size([48, 7, 3, 224, 224]))

Create Patch-level Data Loaders

In [47]:
dataloader_train = create_data_loader(resized_tensor_train, field_numbers_train, batch_size=config.batch_size, shuffle=True)
dataloader_eval = create_data_loader(resized_tensor_eval, field_numbers_eval, batch_size=config.batch_size, shuffle=False)

### Extract Features

In [48]:
device = 'cuda'
vit_extractor = VisionTransformerExtractor()

tf_features_train, train_coord_dl = extract_features(vit_extractor, dataloader_train, device)
tf_features_eval, eval_coord_dl = extract_features(vit_extractor, dataloader_eval, device)

print("Vision Transformer (trained on imagenet) Extracted Features Shape:", tf_features_train.shape[1])  

Vision Transformer (trained on imagenet) Extracted Features Shape: 768


### K-means

In [52]:
kmeans = kmeans_function(tf_features_train.cpu(), n_clusters=2, random_state=14)

train_patch_predictions = kmeans.predict(tf_features_train.reshape(tf_features_train.size(0), -1).numpy().astype(np.float32))
eval_patch_predictions = kmeans.predict(tf_features_eval.reshape(tf_features_eval.size(0), -1).numpy().astype(np.float32))

### Evaluation

Ensure all predictions are not zeros/ones

In [53]:
np.unique(eval_patch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy: Convert sub-patch level labels to patch-level labels and compare with ground truth

In [54]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics_patch_level(field_numbers_eval, eval_patch_predictions, config.labels_path)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 1
Accuracy: 52.46
Precision: 57.14
Recall: 68.57
F1-score: 62.34
F2-score: 65.93


Save Model

In [57]:
# with open(config.vit_imagenet_path, 'wb') as file:
#     pickle.dump(kmeans, file)